In [1]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import warnings
import time
from collections import Counter
warnings.filterwarnings("ignore")


plt.style.use("fivethirtyeight")

sns.set_style({'font.sans-serif': ['simsun', 'Arial']})
sns.set_style('darkgrid', {'font.sans-serif': ['simhei', 'Arial']})
%matplotlib inline

In [2]:
res = {}

# 年报-对外投资信息

In [3]:
df_nianbao_touzi_bkp = pd.read_excel(r'../../Data/CompInfo/年报-对外投资信息.xlsx')

In [4]:
df_nianbao_touzi = df_nianbao_touzi_bkp.copy()

In [5]:
df_nianbao_touzi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93369 entries, 0 to 93368
Data columns (total 4 columns):
企业编号    93369 non-null int64
投资金额    1456 non-null float64
投资占比    1456 non-null float64
年报年份    93367 non-null float64
dtypes: float64(3), int64(1)
memory usage: 2.8 MB


In [6]:
df_nianbao_touzi_clean = df_nianbao_touzi.loc[(
    df_nianbao_touzi['投资金额'].notnull())]

agg_func = {
    '投资金额': ['sum', 'min', 'max'],
    '企业编号': ['count']
}
df_nianbao_touzi_group = df_nianbao_touzi_clean.groupby(
    ['企业编号']).agg(agg_func)
df_nianbao_touzi_group.columns = [
    '_'.join(col).strip() for col in df_nianbao_touzi_group.columns.values]
df_nianbao_touzi_group['企业编号'] = df_nianbao_touzi_group.index
df_nianbao_touzi_group.rename(columns={'企业编号_count': '投资次数'}, inplace=True)
df_nianbao_touzi_group.head(n=2)

,投资金额_sum,投资金额_min,投资金额_max,投资次数,企业编号
企业编号,,,,,
1028,3703345.70,11600.0,3208193.70,7,1028
1134,170384.71,3.0,25505.71,56,1134


In [7]:
res['年报-对外投资信息'] = df_nianbao_touzi_group

# 年报-股东股权转让

In [8]:
df_nianbao_gudongguquanzhuanrang_bkp = pd.read_excel(
    r'../../Data/CompInfo/年报-股东股权转让.xlsx')

In [11]:
df_nianbao_gudongguquanzhuanrang = df_nianbao_gudongguquanzhuanrang_bkp.copy()
df_nianbao_gudongguquanzhuanrang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6855 entries, 0 to 6854
Data columns (total 5 columns):
企业编号       6855 non-null int64
变更前股权比例    5010 non-null object
变更后股权比例    5015 non-null object
股权变更日期     5022 non-null object
年报年份       6849 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 267.9+ KB


In [12]:
df_nianbao_gudongguquanzhuanrang['变更前股权比例'] = df_nianbao_gudongguquanzhuanrang['变更前股权比例'].str.rstrip(
    '%').astype(float)
df_nianbao_gudongguquanzhuanrang_clean = df_nianbao_gudongguquanzhuanrang.loc[(
    df_nianbao_gudongguquanzhuanrang['变更后股权比例'].notnull())]
df_nianbao_gudongguquanzhuanrang_clean['变更后股权比例'] = df_nianbao_gudongguquanzhuanrang_clean[~df_nianbao_gudongguquanzhuanrang_clean["变更后股权比例"].str.contains(
    '-')]['变更后股权比例'].str.rstrip('%').astype(float)

df_nianbao_gudongguquanzhuanrang_clean['变更股权比例差'] = df_nianbao_gudongguquanzhuanrang_clean['变更后股权比例'] - \
    df_nianbao_gudongguquanzhuanrang_clean['变更前股权比例']
# df_nianbao_gudongguquanzhuanrang_clean.head(n=3)
agg_func = {
    '变更股权比例差': ['sum', 'max', 'min'],
    '变更前股权比例': ['sum'],
    '变更后股权比例': ['sum'],
    '企业编号': ['count']
}
df_nianbao_gudongguquanzhuanrang_clean_new = df_nianbao_gudongguquanzhuanrang_clean.groupby([
                                                                                            '企业编号']).agg(agg_func)
df_nianbao_gudongguquanzhuanrang_clean_new.columns = [
    '_'.join(col) for col in df_nianbao_gudongguquanzhuanrang_clean_new.columns.values]
df_nianbao_gudongguquanzhuanrang_clean_new.columns = [
    '变更股权比例差_sum', '变更股权比例差_max', '变更股权比例差_min', '变更前股权比例_sum', '变更后股权比例_sum', '变更次数']
df_nianbao_gudongguquanzhuanrang_clean_new['企业编号'] = df_nianbao_gudongguquanzhuanrang_clean_new.index
df_nianbao_gudongguquanzhuanrang_clean_new.head(n=3)

,变更股权比例差_sum,变更股权比例差_max,变更股权比例差_min,变更前股权比例_sum,变更后股权比例_sum,变更次数,企业编号
企业编号,,,,,,,
1001,5.24,2.60,-0.80,90.35,95.59,11,1001
1002,-10.91,-10.91,-10.91,61.96,51.05,1,1002
1013,-13.89,1.11,-15.00,43.90,30.01,2,1013


In [13]:
res['年报-股东股权转让'] = df_nianbao_gudongguquanzhuanrang_clean_new

# 工商基本信息表

In [14]:
df_gongshangxinxi_bkp = pd.read_excel(r'../../Data/CompInfo/工商基本信息表.xlsx')

In [74]:
df_gongshangxinxi = df_gongshangxinxi_bkp.copy()
df_gongshangxinxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 19 columns):
企业编号          3000 non-null int64
注册资本（万元）      3000 non-null float64
注册资本币种(正则)    3000 non-null object
成立日期          3000 non-null object
经营状态          3000 non-null object
行业大类（代码）      2999 non-null object
行业小类（代码）      2999 non-null object
类型            3000 non-null object
省份代码          3000 non-null int64
城市代码          3000 non-null int64
地区代码          3000 non-null int64
是否上市          3000 non-null object
经营期限自         2987 non-null object
经营期限至         521 non-null object
登记机关区域代码      2992 non-null float64
发照日期          2979 non-null object
员工人数          2986 non-null float64
注销原因          227 non-null object
注销时间          1 non-null object
dtypes: float64(3), int64(4), object(12)
memory usage: 445.4+ KB


In [75]:
df_gongshangxinxi['经营状态'].value_counts()

存续（在营、开业、在册）    1762
在营（开业）企业         401
存续               362
开业               343
在业                63
存续(在营、开业、在册)      48
迁出                21
Name: 经营状态, dtype: int64

In [76]:
df_gongshangxinxi['经营状态'] = df_gongshangxinxi['经营状态'].str.replace(
    '存续（在营、开业、在册）', '开业')
df_gongshangxinxi['经营状态'] = df_gongshangxinxi['经营状态'].str.replace(
    '在业', '开业')
df_gongshangxinxi['经营状态'] = df_gongshangxinxi['经营状态'].str.replace(
    '在营（开业）企业', '开业')
df_gongshangxinxi['经营状态'].value_counts()

开业              2569
存续               362
存续(在营、开业、在册)      48
迁出                21
Name: 经营状态, dtype: int64

In [77]:
zhuangtai = df_gongshangxinxi.groupby(['企业编号', '经营状态'])[
    '经营状态'].count().unstack().fillna(0).astype(int)
zhuangtai['企业编号'] = zhuangtai.index
zhuangtai['开业'] = zhuangtai['存续']+zhuangtai['存续(在营、开业、在册)']+zhuangtai['开业']
zhuangtai = pd.DataFrame(
    {'企业编号': zhuangtai['企业编号'], '经营状态_开业': zhuangtai['开业'], '经营状态_迁出': zhuangtai['迁出']})
res['经营状态'] = zhuangtai

In [78]:
df_hangye = df_gongshangxinxi.groupby(['企业编号', '行业大类（代码）'])[
    '行业大类（代码）'].count().unstack().fillna(0).astype(int)
df_hangye.columns = ['行业大类_'+col for col in df_hangye.columns.values]
df_hangye['企业编号'] = df_hangye.index
res['行业大类'] = df_hangye

In [79]:
df_gongshangxinxi['是否上市'] = df_gongshangxinxi['是否上市'].str.replace('Y', '1')
df_gongshangxinxi['是否上市'] = df_gongshangxinxi['是否上市'].str.replace('N', '0')
df_gongshangxinxi['是否上市'] = df_gongshangxinxi['是否上市'].astype(int)

In [80]:
df_gongshangxinxi.loc[df_gongshangxinxi['注册资本币种(正则)'] == '美元', [
    '注册资本（万元）']] = df_gongshangxinxi.loc[df_gongshangxinxi['注册资本币种(正则)'] == '美元', ['注册资本（万元）']]*8.2777
df_gongshangxinxi.loc[df_gongshangxinxi['注册资本币种(正则)'] == '美元']

,企业编号,注册资本（万元）,注册资本币种(正则),成立日期,经营状态,行业大类（代码）,行业小类（代码）,类型,省份代码,城市代码,地区代码,是否上市,经营期限自,经营期限至,登记机关区域代码,发照日期,员工人数,注销原因,注销时间
1309,2310,17.429525,美元,2000-05-15,开业,C,29,外企,44,4406,440600,1,2000-05-15,2015-05-14,440600.0,2008-05-19,14.0,NaN,NaN


In [81]:
temp = pd.DataFrame({'企业编号': df_gongshangxinxi['企业编号'],
                     '注册资本（万元）': df_gongshangxinxi['注册资本（万元）'],
                     '是否上市': df_gongshangxinxi['是否上市']})
res['merge_注册资本_是否上市'] = temp

In [82]:
df_gongshangxinxi['成立日期'] = pd.to_datetime(df_gongshangxinxi['成立日期'])

In [83]:
df_gongshangxinxi['成立年数'] = round(abs(df_gongshangxinxi['成立日期'].sub(pd.to_datetime(
    time.strftime('%Y-%m-%d', time.localtime(time.time()))), axis=0).dt.days)/365).astype(int)

In [101]:
df_gongshangxinxi.head(n=3)

,企业编号,注册资本（万元）,注册资本币种(正则),成立日期,经营状态,行业大类（代码）,行业小类（代码）,类型,省份代码,城市代码,...,经营期限自,经营期限至,登记机关区域代码,发照日期,员工人数,注销原因,注销时间,成立年数,是否注销,是否外企
0,1001,746.138857,人民币,1992-10-19,开业,J,66,民营,31,3100,...,1992-10-19,NaN,310100.0,1992-10-19,19.0,NaN,NaN,26,0,NaN
1,1002,82.772800,人民币,2000-09-19,开业,O,80,民营,44,4400,...,2000-09-19,NaN,440000.0,2017-12-21,12.0,NaN,NaN,19,0,NaN
2,1003,80.000000,人民币,1999-07-21,开业,C,36,民营,42,4200,...,1999-07-21,NaN,420000.0,2018-10-22,11.0,NaN,NaN,20,0,NaN


In [86]:
df_gongshangxinxi['是否注销'] = df_gongshangxinxi['注销原因'].fillna(0)
df_gongshangxinxi.loc[(df_gongshangxinxi['注销原因'].notnull()), '是否注销'] = 1

In [98]:
waiqi = df_gongshangxinxi.groupby(['企业编号', '类型'])['类型'].count(
).unstack().fillna(0).astype(int)
waiqi['企业编号'] = waiqi.index

In [104]:
res['merge_成立年数_是否注销_是否外企'] = pd.DataFrame({
    '是否注销': df_gongshangxinxi['是否注销'],
    '是否外企': waiqi['外企'],
    '企业编号': waiqi['企业编号'],
    '成立年数': df_gongshangxinxi['成立年数']
})

# 年报企业基本信息

In [105]:
df_qiyejiben_bkp = pd.read_excel(r'../../Data/CompInfo/年报-企业基本信息.xlsx')

In [106]:
df_qiyejiben = df_qiyejiben_bkp.copy()

In [107]:
df_qiyejiben.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15056 entries, 0 to 15055
Data columns (total 10 columns):
企业编号                  15056 non-null int64
注册资本                  1750 non-null object
企业经营状态                14837 non-null object
从业人数                  14838 non-null object
是否有网站或网点              14838 non-null object
企业是否有投资信息或购买其他公司股权    14835 non-null object
有限责任公司本年度是否发生股东股权转    14836 non-null object
是否提供对外担保              14827 non-null object
发布日期                  15039 non-null object
年报年份                  15048 non-null float64
dtypes: float64(1), int64(1), object(8)
memory usage: 1.1+ MB


In [108]:
df_qiyejiben['企业经营状态'].value_counts()

开业         13590
正常开业         739
开业/正常经营      420
存续            64
其他            10
歇业             7
清算             6
停业             1
Name: 企业经营状态, dtype: int64

In [113]:
jingyingzhaungtai = df_qiyejiben.groupby(['企业编号', '企业经营状态'])[
    '企业经营状态'].count().unstack().fillna(0).astype(int)
jingyingzhaungtai['企业编号'] = jingyingzhaungtai.index
jingyingzhaungtai['开业'] = jingyingzhaungtai['开业'] + \
    jingyingzhaungtai['开业/正常经营'] + \
    jingyingzhaungtai['正常开业']+jingyingzhaungtai['存续']
res['企业基本信息经营状态'] = pd.DataFrame({
    '企业编号': jingyingzhaungtai['企业编号'],
    '停业': jingyingzhaungtai['停业'],
    '其他': jingyingzhaungtai['其他'],
    '开业': jingyingzhaungtai['开业'],
    '歇业': jingyingzhaungtai['歇业'],
    '清算': jingyingzhaungtai['清算'],
})

# 年报-的对外提供保证担保信息

In [114]:
df_duiwaidanbao_bkp = pd.read_excel(r'../../Data/CompInfo/年报-的对外提供保证担保信息.xlsx')

In [115]:
df_duiwaidanbao = df_duiwaidanbao_bkp.copy()

In [116]:
df_duiwaidanbao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10875 entries, 0 to 10874
Data columns (total 8 columns):
企业编号       10875 non-null int64
主债权数额      8777 non-null object
保证担保的范围    594 non-null object
保证的期间      8751 non-null object
保证的方式      8754 non-null object
主债权种类      8746 non-null object
履行债务的期限    10864 non-null object
年报年份       10875 non-null int64
dtypes: int64(2), object(6)
memory usage: 679.8+ KB


In [122]:
tmpdf = pd.DataFrame()
tmpdf['担保次数'] = df_duiwaidanbao.groupby(['企业编号'])['企业编号'].count()
tmpdf['企业编号'] = tmpdf.index
res = {'企业担保次数': tmpdf}

# 年报-网站或网点信息

In [124]:
df_wangzhan_bkp = pd.read_excel(r'../../Data/CompInfo/年报-网站或网点信息.xlsx')

In [125]:
df_wangzhan = df_wangzhan_bkp.copy()

In [126]:
df_wangzhan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13553 entries, 0 to 13552
Data columns (total 2 columns):
企业编号    13553 non-null int64
年报年份    13546 non-null float64
dtypes: float64(1), int64(1)
memory usage: 211.8 KB


In [127]:
tmpdf = pd.DataFrame()
tmpdf['网站或网点数'] = df_wangzhan.groupby(['企业编号'])['企业编号'].count()
tmpdf['企业编号'] = tmpdf.index
res = {'网站或网点数': tmpdf}

# 债券信息

In [129]:
df_zaiquan_bkp = pd.read_excel(r'../../Data/CompInfo/债券信息.xlsx')

In [130]:
df_zaiquan = df_zaiquan_bkp.copy()

In [131]:
df_zaiquan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 14 columns):
企业编号          2400 non-null int64
债券信用评级        386 non-null object
债券期限          2314 non-null object
债券品种          2400 non-null object
发行日期          2400 non-null object
付息日期          2032 non-null object
兑付日期          2313 non-null object
计划发行总额（亿元）    2400 non-null float64
实际发行总额（亿元）    35 non-null float64
币种            2400 non-null object
流通场所          17 non-null object
票面利率（%）       2399 non-null float64
主体信用评级        382 non-null object
付息方式          2400 non-null object
dtypes: float64(3), int64(1), object(10)
memory usage: 262.6+ KB


In [132]:
df_zaiquan['债券信用评级'].value_counts()

AAA    228
AA+     86
AA      67
B        2
AA-      1
C        1
CC       1
Name: 债券信用评级, dtype: int64

In [143]:
xinyongpingji = df_zaiquan.groupby(['企业编号', '债券信用评级'])[
    '债券信用评级'].count().unstack().fillna(0).astype(int)
xinyongpingji.columns = ['债券信用评级_'+col for col in xinyongpingji.columns.values]
xinyongpingji['企业编号'] = xinyongpingji.index
res = {'债券信用评级': xinyongpingji}
# xinyongpingji

In [157]:
fuxifangshi = df_zaiquan.groupby(['企业编号', '付息方式'])[
    '付息方式'].count().unstack().fillna(0).astype(int)
fuxifangshi.columns = ['付息方式_'+col for col in fuxifangshi.columns.values]
fuxifangshi['企业编号'] = fuxifangshi.index
res = {'付息方式': fuxifangshi}

In [153]:
zaiquanzhonglei = df_zaiquan.groupby(['企业编号', '债券品种'])[
    '债券品种'].count().unstack().fillna(0).astype(int)
zaiquanzhonglei.columns = [
    '债券品种_'+col for col in zaiquanzhonglei.columns.values]
zaiquanzhonglei['企业编号'] = zaiquanzhonglei.index
res = {'债券品种': zaiquanzhonglei}
# zaiquanzhonglei

In [161]:
df_zaiquan['主体信用评级'] = df_zaiquan['主体信用评级'].str.rstrip('-')

In [168]:
# df_zaiquan['主体信用评级']
zhutipingji = df_zaiquan.groupby(['企业编号', '主体信用评级'])[
    '主体信用评级'].count().unstack().fillna(0).astype(int)
zhutipingji.columns = ['主体信用评级_'+col for col in zhutipingji.columns.values]
zhutipingji['企业编号'] = zhutipingji.index
zhutipingji.drop(['主体信用评级_'], axis=1)
# zhutipingji.head(n=4)
# new_zhutipingji
res = {'主体信用评级': pd.DataFrame({
    '主体信用评级_A': zhutipingji['主体信用评级_A'],
    '主体信用评级_AA': zhutipingji['主体信用评级_AA'],
    '主体信用评级_AA+': zhutipingji['主体信用评级_AA+'],
    '主体信用评级_AAA': zhutipingji['主体信用评级_AAA'],
    '企业编号': zhutipingji['企业编号'],
})}